<a href="https://colab.research.google.com/github/utkarsh-in/ELSS-correlation-study/blob/main/ELSS_correlation_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this notebook is to find commonality between ELSS mutual funds so that you can increase diversification when choosing more than one ELSS fund to invest in.

The data is scraped in real-time from www.moneycontrol.com.

We take a few assumptions (though the code is made s.t. the assumptions can reveresed with minimum changes):
1. Ypu are looking at only the category of the stock holdings and not the stock itself.
2. Default first choice is Quant Mutual fund due to its high returns compared to its peers.

Conclusion:
Canara robero elss fund had the lowest commality (~41%) and thus is the best choice for diversification.

You can also find the experimental code at the end of the file.

In [ ]:
!pip install html-table-parser-python3

In [ ]:
import pandas as pd
from html_table_parser.parser import HTMLTableParser
import urllib.request

In [ ]:
QUANT_URL = "https://www.moneycontrol.com/mutual-funds/quant-tax-plan-direct-plan/portfolio-holdings/MES037"
CANARA_URL = "https://www.moneycontrol.com/mutual-funds/canara-robeco-equity-tax-saver-fund-direct-plan/portfolio-overview/MCA200"
MIRAE_URL = "https://www.moneycontrol.com/mutual-funds/mirae-asset-tax-saver-fund-regular-plan/portfolio-overview/MMA148"
IDFC_URL = "https://www.moneycontrol.com/mutual-funds/idfc-tax-advantage-elss-fund-direct-plan/portfolio-overview/MAG741"
PGIM_URL = "https://www.moneycontrol.com/mutual-funds/pgim-india-elss-tax-saver-fund-direct-plan-/portfolio-overview/MPA163"
PARAG_URL = "https://www.moneycontrol.com/mutual-funds/parag-parikh-tax-saver-fund-regular-plan/portfolio-overview/MPP011"
DSP_URL = "https://www.moneycontrol.com/mutual-funds/dsp-tax-saver-fund-regular/portfolio-overview/MDS060"

In [ ]:
def url_get_contents(url):

	# Opens a website and read its
	# binary contents (HTTP Response Body)

	#making request to the website
	req = urllib.request.Request(url=url)
	f = urllib.request.urlopen(req)

	#reading contents of the website
	return f.read()

In [ ]:
def get_data(url):
  # defining the html contents of a URL.
  xhtml = url_get_contents(url).decode('utf-8')

  # Defining the HTMLTableParser object
  p = HTMLTableParser()

  # feeding the html contents in the
  # HTMLTableParser object
  p.feed(xhtml)

  return pd.DataFrame(p.tables[3])

In [ ]:
def pre_process(data):
  data = data.rename(columns=data.iloc[0]).drop(data.index[0])
  data = data[['Sector', '% of Total Holdings']]
  data['% of Total Holdings'] = data['% of Total Holdings'].str.rstrip('%').astype('float')
  quant = data.groupby('Sector').sum()
  return data

In [ ]:
def find_common_invest(data1, data2):
  percent_common = 0
  categories = set(data1.index).union(set(data2.index))

  for x in categories:
    if (x in data1['% of Total Holdings'] and x in data2['% of Total Holdings']):
      percent_common += min(data1['% of Total Holdings'][x], data2['% of Total Holdings'][x])

  return percent_common

In [ ]:
def find_common_invest_by_category(url1, url2=QUANT_URL):
  data1 = pre_process(get_data(url1))
  data2 = pre_process(get_data(url2))

  print(find_common_invest(data1, data2))


Results:

In [ ]:
find_common_invest_by_category(CANARA_URL, QUANT_URL)

41.980000000000004


In [ ]:
find_common_invest_by_category(MIRAE_URL, QUANT_URL)

44.22


In [ ]:
find_common_invest_by_category(IDFC_URL)

41.01


In [ ]:
find_common_invest_by_category(PGIM_URL)

46.99999999999999


In [ ]:
find_common_invest_by_category(PARAG_URL)

49.51


In [ ]:
find_common_invest_by_category(DSP_URL)

43.7


Experimental code after this 


In [ ]:
url = QUANT_URL

In [ ]:
data = get_data(url)
def pre_process(data):
  data = data.rename(columns=data.iloc[0]).drop(data.index[0])
  data = data[['Sector', '% of Total Holdings']]
  data['% of Total Holdings'] = data['% of Total Holdings'].str.rstrip('%').astype('float')
  quant = data.groupby('Sector').sum()
  return data

,Sector,% of Total Holdings
1,Banks,7.41
2,Cigarettes,6.62
3,Banks,6.24
4,Consumer food,6.18
5,Trading,5.37
6,"Engineering, designing, construction",4.33
7,Port,4.12
8,Refineries/marketing,3.47
9,Oil exploration,3.38
10,Cement,3.32


In [ ]:
# READ AND PREPROCESS DATA

data = pd.read_csv('ELSS.csv')
data['percent_quant'] = data['percent_quant'].str.rstrip('%').astype('float')
data['percent_canara'] = data['percent_canara'].str.rstrip('%').astype('float')

data.head()

,stocks_quant,sector_quant,percent_quant,stocks_canara,sector_canara,percent_canara
0,State Bank Of India,Banks,7.41,ICICI Bank Ltd.,Banks,7.39
1,ITC Limited,Cigarettes,6.62,Reliance Industries Ltd.,Refineries/marketing,7.00
2,ICICI Bank Ltd.,Banks,6.24,Infosys Ltd.,Computers - software,6.95
3,Ruchi Soya Industries Ltd.,Consumer food,6.18,HDFC Bank Ltd.,Banks,6.68
4,Adani Enterprises Ltd.,Trading,5.37,State Bank Of India,Banks,3.03


In [ ]:
quant = data[['sector_quant', 'percent_quant']].groupby('sector_quant').sum()
canara = data[['sector_canara', 'percent_canara']].groupby('sector_canara').sum()

In [ ]:
canara

,percent_canara
sector_canara,
Aluminium,0.86
Auto ancillaries,2.32
Banks,21.66
Bearings,2.12
Brew/distilleries,1.97
Cement,2.25
Chemicals - inorganic,1.70
Chemicals - speciality,1.01
Computers - software,11.00


In [ ]:
# FIND % COMMON STOCKS BY CATEGORY

percent_common = 0
categories = set(quant.index).union(set(canara.index))

for x in categories:
  if (x in canara['percent_canara'] and x in quant['percent_quant']):
    percent_common += min(canara['percent_canara'][x], quant['percent_quant'][x])

percent_common

40.79

{'Aluminium',
 'Auto ancillaries',
 'Banks',
 'Bearings',
 'Brew/distilleries',
 'Cement',
 'Chemicals - inorganic',
 'Chemicals - speciality',
 'Cigarettes',
 'Commercial vehicles',
 'Computers - software',
 'Construction civil',
 'Consumer food',
 'Consumer products',
 'Copper & copper products',
 'Diversified',
 'Engineering, designing, construction',
 'Fabrics and garments',
 'Gas transmission/marketing',
 'Gems, jewellery and watches',
 'Hospital',
 'Hotels/resorts',
 'Housing finance',
 'Industrial electronics',
 'Industrial gas',
 'Insurance',
 'It enabled services – software',
 'Leather and leather products',
 'Logistics solution provider',
 'Lpg/cng/png/lng supplier',
 'Motor cycles/scooters',
 'Nbfc',
 'Oil exploration',
 'Other financial services',
 'Paints',
 'Passenger/utility vehicles',
 'Personal care',
 'Pesticides and agrochemicals',
 'Pharmaceuticals',
 'Port',
 'Power',
 'Power equipment',
 'Refineries/marketing',
 'Residential/commercial/sez project',
 'Retailing',


In [ ]:
# SCRAPING LATEST DATA



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Library for opening url and creating
# requests


# pretty-print python data structures
from pprint import pprint

# for parsing all the tables present
# on the website


# for converting the parsed data in a
# pandas dataframe
import pandas as pd


# Opens a website and read its
# binary contents (HTTP Response Body)




[['Particulars', 'This Scheme', 'Category'],
 ['No. of Stocks', '52', '47.54'],
 ['Top 5 Stock Weight', '31.82%', '30.29%'],
 ['Top 10 Stock Weight', '50.44%', '47.8%'],
 ['Top 3 Sector Weight', '31.81%', '38.88%']]


PANDAS DATAFRAME

                     0            1         2
0          Particulars  This Scheme  Category
1        No. of Stocks           52     47.54
2   Top 5 Stock Weight       31.82%    30.29%
3  Top 10 Stock Weight       50.44%     47.8%
4  Top 3 Sector Weight       31.81%    38.88%


In [ ]:
print(pd.DataFrame(p.tables[3]))

                                             0  \
0                            Stock Invested in   
1                          State Bank Of India   
2                                  ITC Limited   
3                              ICICI Bank Ltd.   
4                   Ruchi Soya Industries Ltd.   
5                       Adani Enterprises Ltd.   
6                           Larsen Toubro Ltd.   
7   Adani Ports And Special Economic Zone Ltd.   
8                     Reliance Industries Ltd.   
9             Oil Natural Gas Corporation Ltd.   
10                         Ambuja Cements Ltd.   

                                       1          2                    3  \
0                                 Sector  Value(Mn)  % of Total Holdings   
1                                  Banks     1015.7                7.41%   
2                             Cigarettes      907.1                6.62%   
3                                  Banks      855.3                6.24%   
4                  

In [ ]:
df = get_data(CANARA_URL)

In [ ]:
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

In [ ]:
df[['Sector', '% of Total Holdings']]

,Sector,% of Total Holdings
1,Banks,7.41%
2,Cigarettes,6.62%
3,Banks,6.24%
4,Consumer food,6.18%
5,Trading,5.37%
6,"Engineering, designing, construction",4.33%
7,Port,4.12%
8,Refineries/marketing,3.47%
9,Oil exploration,3.38%
10,Cement,3.32%


In [ ]:
for x in df.columns:
  print(x)

0
1
2
3
4
5
6
7
8
